# Hidden Markov Model

Tal cual como redes Bayesianas con probabilidades discretas, también es posible estudiar secuencias temporales discretas. Estas secuencias se dan en muchas problemas en que necesitamos clasificar el estado de un proceso (oculto o latente) $Z$ que toma valores (discretos o contínuos) a los que tenemos acceso a través de  variables observadas $X$.  


Sin embargo, si sabemos que el proceso $Z$ tiene la propiedad de Markov de primer orden $p(z_k \vert z_{k-1},\ldots,z_1)=p(z_k \vert z_{k-1})$ y que las observaciones $X$ son independientes entre si y solamente dependen de la variable oculta.

Las variables observadas pueden ser variables discretas o continuas. Cuando se consideran v.a. discretas, podemos especificar una matriz $\boldsymbol{A}_{[N \times N]}$ de transición para los $N$ posibles estados, una matriz de emisión $\boldsymbol{B}_{[M \times N]}$ y un conjunto de probabilidades iniciales $\boldsymbol{\pi}$.

$
\begin{align*}
\boldsymbol{A}_{n,n} = 
\begin{pmatrix}
z_{1,1} & z_{1,2} & \cdots & z_{1,N} \\
z_{2,1} & z_{2,2} & \cdots & z_{2,N} \\
\vdots  & \vdots  & \ddots & \vdots  \\
z_{N,1} & z_{N,2} & \cdots & z_{N,N} 
\end{pmatrix};\quad
&\boldsymbol{B}_{m,n} = 
\begin{pmatrix}
x_{1,1} & x_{1,2} & \cdots & x_{1,N} \\
x_{2,1} & x_{2,2} & \cdots & x_{2,N} \\
\vdots  & \vdots  & \ddots & \vdots  \\
x_{M,1} & x_{M,2} & \cdots & x_{M,N} 
\end{pmatrix}; \quad
\boldsymbol{\pi}=[\pi_1,\ldots,\pi_N]
\end{align*}
$

Dado un modelo $\lambda=(\boldsymbol{A,B,\pi})$, una secuencia observada $X_n=\{x_1,\ldots,x_n\}$ y un conjunto de estados $Z_n=\{z_1,\ldots,z_n\}$, los patrones de inferencia para HMM son los siguientes:
	
Filtrado : Determinar $p(z_n \vert X_n)$.

Verosimilitud :  Determinar $p(X_n)$. 

Secuencia más probable :  $p(Z_n \vert X_n)$.

In [320]:
import numpy as np

class HMM:

    def __init__(self,A,B,pi):
        self.A = A
        self.B = B
        self.pi = pi

    def alpha_recursion(self,observations):
        D=len(self.pi)
        T=len(observations)
        alpha=np.zeros((D,T))
        alpha[:,0]=self.pi*self.B[observations[0]]
        #alpha[:,0]=alpha[:,0]/alpha[:,0].sum()
        for i in range(1,T):
            alpha[:,i]=np.dot(np.diag(self.B[observations[i]]),np.dot(self.A,alpha[:,i-1]))
            #alpha[:,i]=alpha[:,i]/alpha[:,i].sum()
        return alpha
    
    def beta_recursion(self,observations):
        D=len(self.pi)
        T=len(observations)
        beta=np.zeros((D,T))
        beta[:,-1]=np.ones(D)
        for i in range(T-1,0,-1):
            beta[:,i-1]=np.dot(self.A,np.diag(self.B[observations[i]]).dot(beta[:,i]))
        return beta
    
    def likelihood(self,observations):
        alpha=self.alpha_recursion(observations)
        likelihood=np.sum(alpha,0)
        return likelihood.sum()
    


In [321]:
B=np.array([[0.7,0.4,0.8],
           [0.3,0.6,0.2]])
A=np.array([[0.5,0.3,0.2],
            [0.0,0.6,0.4],
            [0.0,0.0,1.0]]).T
pi=np.array([0.9,0.1,0.0])
observations=np.array([0,1,0])

hmm=HMM(A,B,pi)


In [322]:
alpha=hmm.alpha_recursion(observations)
beta=hmm.beta_recursion(observations)

print('p(v_1,v_2,v_3)={0:0.3f}'.format(alpha[:,-1].sum()))

p(v_1,v_2,v_3)=0.154


In [323]:
alpha_norm=alpha/alpha.sum(axis=0)

for i in range(alpha_norm.shape[1]):
    print('alpha {0} : {1}'.format(i,alpha_norm[:,i]))

alpha 0 : [0.94029851 0.05970149 0.        ]
alpha 1 : [0.37694456 0.50977264 0.11328281]
alpha 2 : [0.21501986 0.2731191  0.51186104]


In [324]:
beta_norm=beta/beta.sum(axis=0)

for i in range(beta_norm.shape[1]):
    print('beta {0} : {1}'.format(i,beta_norm[:,i]))

beta 0 : [0.08823529 0.32521008 0.58655462]
beta 1 : [0.18421053 0.23684211 0.57894737]
beta 2 : [0.33333333 0.33333333 0.33333333]


In [325]:
pv1=alpha_norm[:,0]*beta_norm[:,0]
pv1=pv1/pv1.sum()
print('p(h_1 |v_1, v_2,v_3)={0}'.format(pv1))

p(h_1 |v_1, v_2,v_3)=[0.81036384 0.18963616 0.        ]


In [326]:
B=np.array([[0.9,0.2],
           [0.1,0.8]])
A=np.array([[0.7,0.3],
            [0.3,0.7]])
pi=np.array([0.5,0.5])
observations=np.array([0,0,1,0,0])

hmm=HMM(A,B,pi)

In [327]:
alpha=hmm.alpha_recursion(observations)
beta=hmm.beta_recursion(observations)


In [328]:
alpha_norm=alpha/alpha.sum(axis=0)

for i in range(alpha_norm.shape[1]):
    print('alpha {0} : {1}'.format(i,alpha_norm[:,i]))

alpha 0 : [0.81818182 0.18181818]
alpha 1 : [0.88335704 0.11664296]
alpha 2 : [0.19066794 0.80933206]
alpha 3 : [0.730794 0.269206]
alpha 4 : [0.86733889 0.13266111]


In [330]:
beta_norm=beta/beta.sum(axis=0)

for i in range(beta_norm.shape[1]):
    print('beta {0} : {1}'.format(i,beta_norm[:,i]))

beta 0 : [0.5923176 0.4076824]
beta 1 : [0.37626718 0.62373282]
beta 2 : [0.65334282 0.34665718]
beta 3 : [0.62727273 0.37272727]
beta 4 : [0.5 0.5]


In [331]:
b1=np.ones(2)
b2=np.dot(A,np.diag(B[observations[4]]).dot(b1))

In [332]:
b3=np.dot(A,np.diag(B[observations[3]]).dot(b2))

In [333]:
b3/b3.sum()

array([0.65334282, 0.34665718])

In [334]:
b4=np.dot(A,np.diag(B[observations[2]]).dot(b3))

In [335]:
b4

array([0.090639, 0.150251])

In [336]:
b4/b4.sum()

array([0.37626718, 0.62373282])